### การเตรียมชุดข้อมูลตัวอย่างของแอปเปิ้ลสายพันธุ์ Gala ในประเทศ UK

- ทำการอ่านไฟล์ Apple462.csv 
- คัดกรองชุดข้อมูลตัวอย่างเฉพาะประเทศ UK

In [1]:
import pandas as pd

df = pd.read_csv('../Apple462.csv')
df = df.drop(df[df["region"] != "UK"].index)
df = df.drop(columns=["Unnamed: 0"])

df.head(10)

,Date,Envi,Fuji,Gala,region
156,27/12/2015,30482,2972,5894,UK
157,20/12/2015,25628,2546,7394,UK
158,13/12/2015,27149,2986,6458,UK
159,6/12/2015,35591,12527,4086,UK
160,29/11/2015,24190,2790,6031,UK
161,22/11/2015,,3256,6663,UK
162,15/11/2015,24425,12086,6502,UK
163,8/11/2015,31910,4652,6286,UK
164,1/11/2015,30435,4360,6428,UK
165,25/10/2015,29522,10090,6552,UK


- นำ features ของวันที่และสายพันธุ์ Gala สร้างชุดข้อมูลตัวอย่างใหม่

In [2]:
df_gala = df.drop(columns=["Envi", "Fuji", "region"])
df_gala.head(10)

,Date,Gala
156,27/12/2015,5894
157,20/12/2015,7394
158,13/12/2015,6458
159,6/12/2015,4086
160,29/11/2015,6031
161,22/11/2015,6663
162,15/11/2015,6502
163,8/11/2015,6286
164,1/11/2015,6428
165,25/10/2015,6552


- เปลี่ยนชนิดข้อมูลของวันที่จาก object เป็น date
- นำชุดข้อมูลตัวอย่างมาเรียงชุดข้อมูลตัวอย่างใหม่ตามวันที่จากเก่าไปใหม่

In [3]:
df_gala["Date"] = pd.to_datetime(df_gala["Date"], format="%d/%m/%Y")
df_gala.info()

<class 'pandas.core.frame.DataFrame'>
Index: 169 entries, 156 to 3344
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    169 non-null    datetime64[ns]
 1   Gala    169 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 4.0+ KB


In [4]:
df_gala.sort_values(by="Date", inplace=True)
df_gala["Date"] = df_gala["Date"].dt.strftime("%d/%m/%Y")
df_gala.head(10)

,Date,Gala
207,04/01/2015,2752
206,11/01/2015,3322
205,18/01/2015,2257
204,25/01/2015,2061
203,01/02/2015,2905
202,08/02/2015,
201,15/02/2015,6493
200,22/02/2015,2122
199,01/03/2015,3492
198,08/03/2015,7404


- ตรวจสอลชุดข้อมูลตัวอย่างว่ามี miss value ในวันที่ใดบ้าง

In [5]:
result = df_gala[df_gala['Gala'] == ' ']
result

,Date,Gala
202,08/02/2015,
187,24/05/2015,


- แก้ไข miss value โดยหาค่าเฉลี่ยของ 2 วันก่อนหน้า
- หลังจากแก้ไขแล้วให้เปลี่ยนชนิดข้อมูลของชุดข้อมูลตัวอย่างของ Gala ให้เป็นจำนวนเต็ม ( Integer ) เนื่องจากค่าเฉลี่ยทีได้อาจได้ค่าเป็นทศนิยม ( Float )

In [6]:
import numpy as np

df_gala.loc[202, "Gala"] = np.nan
df_gala.loc[187, "Gala"] = np.nan

df_gala["Gala"] = pd.to_numeric(df_gala["Gala"])
df_gala = df_gala.interpolate()

df_gala["Gala"] = df_gala["Gala"].astype("int64")
df_gala.info()

<class 'pandas.core.frame.DataFrame'>
Index: 169 entries, 207 to 3333
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    169 non-null    object
 1   Gala    169 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 8.0+ KB


- บันทึกไฟล์ csv ของแอปเปิ้ลสายพันธ์ุ Gala เพื่อใช้สำหรับสร้างโมเดลต่อไป

In [7]:
df_gala.to_csv("gala.csv", sep=",", encoding="utf-8", index=False)